## Import and export of data from different sources in GRASS GIS

GRASS GIS Location can contain data only in one coordinate reference system (CRS)
in order to have full control over reprojection
and avoid issues coming from on-the-fly reprojection.


When starting a project, decide which CRS you will use. Create
a new Location using Location Wizard (accessible from GRASS GIS start-up page).
Specify desired CRS either by providing
EPSG code (can be found e.g. at [epsg.io](http://epsg.io/))
or by providing a georeferenced file (such as Shapefile) which has
the CRS you want.

### Importing data in common vector and raster formats


For basic import of raster and vector files, use _r.import_
and _v.import_, respectively.
These modules will reproject the input data if necessary.
If the input data's CRS matches the Location's CRS, we can use
_r.in.gdal_ or _v.in.ogr_
for importing raster and vector.


Alternatively, you can use a two-step approach
for the cases when the data's CRS doesn't match the Location's CRS.
First create a new temporary Location
based on the CRS of the data you want to import, switch to this Location
and then use _r.in.gdal_ or _v.in.ogr_
to import raster and vector data, respectively. Then switch to the Location
of your project and use
_r.proj_ and _v.proj_
to reproject data from the temporary Location to your project Location.
This approach is necessary for formats which are not supported by
_r.import_ and _v.import_ modules.
Modules  _r.proj_ and _v.proj_
can be also used for bringing raster and vector maps from one Location to another.


Modules _r.in.gdal_ and _v.in.ogr_
check whether the CRS of the imported data matches the Location's CRS.
Sometimes the CRS of imported data is not specified correctly
or is missing and therefore import fails.
If you know that the actual CRS matches the Location's CRS,
it is appropriate to use _r.in.gdal_'s
or _v.in.ogr_'s -o flag to overwrite the projection
check and import the data as they are.


If you zoom to raster or vector in GRASS GUI and it does not fit with
the rest of the data, it means that it was imported with wrong projection
information (or with the -o flag when the coordinates in fact don't match).
You can use r.info and v.info to get the information
about the extents of (already imported) rasters and vectors.

### Importing CSV and other ASCII data


There are many formats of plain text files. In the context of GIS we usually
talk about ASCII formats and CSV files. CSV files usually hold only
coordinates and sometimes attributes of points.
These files usually don't have CRS information attached to them,
so we must be very careful and import the data only if the coordinates
are in the CRS of the Location we are using.


Let's create a CSV file called `points.txt`
using a text editor (Notepad++, TextEdit, MS Notepad), for example:

```
637803.6,223804.7
641835.5,223761.2
643056.0,217419.0
```

The coordinates we entered are in EPSG:3358 and we assume that the
GRASS Location is using this CRS as well.
This file can be imported to GRASS GIS using:

In [ ]:
!v.in.ascii input=points.txt output=test_ascii separator=comma x=1 y=2

Notice, we have to specify the column number where the X and Y (optionally Z)
coordinates are stored. In this example, X coordinates are in the first column
Y in the second one. Don't forget to specify correct column delimiter.

If the data are not in the CRS we are using, create a new Location
with matching CRS,
import the data and use _v.proj_ as described above.


### Importing lidar point clouds

Lidar point clouds can be imported in two ways: as raster maps using binning
or as vector points. However, one must explore the dataset first.


In command line, we can check the projection information and other metadata
about a LAS file using _lasinfo_ tool:

In [ ]:
!lasinfo tile_0793_016_spm.las

_r.in.lidar_ module can be used to scan the spatial extent
of the dataset:

In [ ]:
!r.in.lidar input=tile_0793_016_spm.las -s

#### Binning

Before creating the actual raster with elevation, we need to decide the extent
and the resolution we will use for the binning. We can use
_r.in.lidar_ module for that by setting the resolution
directly and using a -e flag to use dataset extent instead of taking it from
the computational region.
We are interested in the density of points, so we use `method=n`:

In [ ]:
!r.in.lidar input=tile_0793_016_spm.las output=tile_0793_016_n method=n -e resolution=2

After determining the optimal resolution for binning and the desired area,
we can use _g.region_ to set the computational region.
_r.in.lidar_ without the additional parameters above
will create a raster map from points using binning with resolution and extent
taken from the computational region:

In [ ]:
!r.in.lidar input=tile_0793_016_spm.las output=tile_0793_016

#### Interpolation

When the result of binning contains a lot of NULL cells or when it is not
smooth enough for further analysis, we can import the point cloud as vector
points and interpolate a raster.


Supposing that we already determined the desired extent and resolution
(using _r.in.lidar_ as described above) we can use
_v.in.lidar_ lidar for import (and using class filter
to get only ground points):

In [ ]:
!v.in.lidar input=tile_0793_016_spm.las output=tile_0793_016 class=2 -r -t -b

This import only the points of class 2 (ground)
in the current computational region
without the attribute table and building the topology.
Then we follow with interpolation using,
e.g. _v.surf.rst_ module:

In [ ]:
!v.surf.rst input=tile_0793_016 elevation=tile_0793_016_elevation slope=tile_0793_016_slope aspect=tile_0793_016_aspect npmin=100 tension=20 smooth=1

#### Importing data in different CRS

In case the CRS of the file doesn't match the CRS
used in the GRASS Location, reprojection can be done before importing
using _las2las_ tool.


The following example command is for reprojecting tiles
in NAD83/North Carolina in feet (EPSG:2264)
into NAD83/North Carolina in meters (EPSG:3358):

In [ ]:
!las2las --a_srs=EPSG:2264 --t_srs=EPSG:3358 -i input_spf.las -o output_spm.las

#### Importing data with broken projection information

Modules _r.in.lidar_ and _v.in.lidar_
check whether the CRS of the imported data matches the Location's CRS.
Sometimes the CRS of imported data is not specified correctly
or is missing and therefore import fails.
If you know that the actual CRS matches the Location's CRS,
it is appropriate to use _r.in.lidar_'s
or _v.in.lidar_'s -o flag to overwrite the projection
check and import the data as they are.

In [ ]:
!r.in.lidar input=tile_0793_016_spm.las -s -o

### Transferring GRASS GIS data between two computers

If two GRASS GIS users want to exchange data, they can use GRASS GIS native
exchange format -- _packed map_. A vector or raster map can be
exported from a GRASS Location in this format using
_v.pack_ or _r.pack_ respectively.
This format preserves everything for a map in a way as it is stored in
a GRASS Database. _Projection of the source and target GRASS Locations
must be the same._


If GRASS GIS users wish to exchange GRASS Mapsets, they can do so as long as
the source and target GRASS Locations have the same projection.
The PERMANENT Mapset should not be usually exchanged as it is a crucial part
of the given Location.
Locations can be easily transferred in between GRASS Database directories
on different computers as they carry all data and projection information
within them and the storage format used in the background is platform independent.
Locations as well as whole GRASS Databases can be copied and moved
in the same way as any other directories on the computer.


### Further resources


    
* [GRASS GIS manual](http://grass.osgeo.org/grass72/manuals)
    
* [About GRASS GIS Database structure](https://grass.osgeo.org/grass72/manuals/grass_database.html)
    
* [GRASS GIS for ArcGIS users](https://grasswiki.osgeo.org/wiki/GRASS_GIS_for_ArcGIS_users)
    
* [epsg.io](http://epsg.io/) (Repository of EPSG codes)

In [ ]:
# end the GRASS session
os.remove(rcfile)